# Modelos Avanzados de Machine Learning para Análisis Futbolístico

## Objetivos del Notebook

Este notebook implementa técnicas avanzadas de machine learning para el análisis predictivo en fútbol, utilizando el dataset "champs" y expandiendo el análisis del Bloque 1 con modelos más sofisticados.

### Contenido del Bloque 2:

1. **Feature Engineering Avanzado**
2. **Implementación de Múltiples Algoritmos**
3. **Evaluación Comparativa de Modelos**
4. **Optimización de Hiperparámetros**
5. **Análisis de Importancia de Variables**
6. **Validación Cruzada y Robustez**
7. **Interpretación de Resultados**
8. **Despliegue de Modelos**

### Algoritmos a Implementar:
- **Regresión Logística** (baseline)
- **Random Forest** (ensemble method)
- **Support Vector Machine** (SVM)
- **Gradient Boosting** (XGBoost)
- **Redes Neuronales** (MLPClassifier)

---

## 1. Importar Librerías y Configuración Inicial

Importamos las librerías necesarias para el análisis avanzado de machine learning.

In [ ]:
# Librerías básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Librerías de machine learning
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.feature_selection import SelectKBest, f_classif, RFE
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, 
                           confusion_matrix, classification_report, roc_auc_score, roc_curve)

# Algoritmos de clasificación
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

# Librerías adicionales
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import joblib
from collections import Counter

# Configuración de visualización
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("✅ Todas las librerías importadas exitosamente")
print(f"📊 Versión de scikit-learn: {sklearn.__version__}")
print(f"🐍 Versión de Python: {sys.version}")
print(f"📅 Fecha de ejecución: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# Cargar datos desde el Bloque 1 o generar datos de ejemplo
print("🔄 Cargando dataset...")

try:
    # Intentar cargar el dataset real
    df = pd.read_csv('../recursos/champs.csv')
    print("✅ Dataset 'champs' cargado desde archivo")
except FileNotFoundError:
    # Generar dataset de ejemplo más completo para demostración
    print("📝 Generando dataset de ejemplo...")
    
    np.random.seed(42)
    n_matches = 1000
    teams = ['Real Madrid', 'Barcelona', 'Atletico Madrid', 'Valencia', 'Sevilla', 
             'Athletic Bilbao', 'Real Sociedad', 'Villarreal', 'Betis', 'Getafe']
    
    # Generar datos base
    data = {
        'match_id': range(1, n_matches + 1),
        'date': pd.date_range('2022-01-01', periods=n_matches, freq='D'),
        'home_team': np.random.choice(teams, n_matches),
        'away_team': np.random.choice(teams, n_matches),
        'home_goals': np.random.poisson(1.3, n_matches),
        'away_goals': np.random.poisson(1.1, n_matches),
        'league': np.random.choice(['La Liga', 'Copa del Rey', 'Champions'], n_matches, p=[0.6, 0.2, 0.2]),
        'season': np.random.choice(['2022-23', '2023-24'], n_matches),
        'matchday': np.random.randint(1, 39, n_matches),
        'attendance': np.random.normal(45000, 15000, n_matches)
    }
    
    df = pd.DataFrame(data)
    
    # Evitar que un equipo juegue contra sí mismo
    same_team_mask = df['home_team'] == df['away_team']
    df.loc[same_team_mask, 'away_team'] = df.loc[same_team_mask, 'home_team'].apply(
        lambda x: np.random.choice([t for t in teams if t != x])
    )

print(f"📊 Dataset cargado con {len(df)} partidos")
print(f"🏆 Equipos únicos: {df['home_team'].nunique()}")
print(f"📅 Rango de fechas: {df['date'].min()} a {df['date'].max()}")

# Mostrar primeras filas
print("\n🔍 Primeras 5 filas del dataset:")
print(df.head())

## 2. Feature Engineering Avanzado

Creamos variables más sofisticadas que capturen patrones complejos en los datos futbolísticos.

In [ ]:
# Feature Engineering Avanzado
print("🛠️ FEATURE ENGINEERING AVANZADO")
print("=" * 50)

# Crear copia del dataframe para trabajar
df_features = df.copy()

# 1. Variables básicas
df_features['total_goals'] = df_features['home_goals'] + df_features['away_goals']
df_features['goal_difference'] = df_features['home_goals'] - df_features['away_goals']
df_features['home_win'] = (df_features['home_goals'] > df_features['away_goals']).astype(int)
df_features['away_win'] = (df_features['home_goals'] < df_features['away_goals']).astype(int)
df_features['draw'] = (df_features['home_goals'] == df_features['away_goals']).astype(int)

# 2. Variables temporales
df_features['date'] = pd.to_datetime(df_features['date'])
df_features['year'] = df_features['date'].dt.year
df_features['month'] = df_features['date'].dt.month
df_features['day_of_week'] = df_features['date'].dt.dayofweek
df_features['is_weekend'] = df_features['day_of_week'].isin([5, 6]).astype(int)

# 3. Variables de contexto
df_features['is_important_match'] = (df_features['league'] == 'Champions').astype(int)
df_features['is_cup_match'] = (df_features['league'] == 'Copa del Rey').astype(int)
df_features['late_season'] = (df_features['matchday'] > 30).astype(int)

# 4. Variables de rendimiento histórico por equipo
def calculate_team_stats(df, team_col, stats_window=5):
    """Calcula estadísticas móviles por equipo"""
    team_stats = {}
    
    for team in df[team_col].unique():
        team_matches = df[df[team_col] == team].sort_values('date')
        
        # Calcular estadísticas móviles
        if team_col == 'home_team':
            goals_for = team_matches['home_goals'].rolling(stats_window, min_periods=1).mean()
            goals_against = team_matches['away_goals'].rolling(stats_window, min_periods=1).mean()
            wins = team_matches['home_win'].rolling(stats_window, min_periods=1).mean()
        else:
            goals_for = team_matches['away_goals'].rolling(stats_window, min_periods=1).mean()
            goals_against = team_matches['home_goals'].rolling(stats_window, min_periods=1).mean()
            wins = team_matches['away_win'].rolling(stats_window, min_periods=1).mean()
        
        # Guardar estadísticas
        for idx, match_id in enumerate(team_matches['match_id']):
            team_stats[match_id] = {
                f'{team_col}_goals_avg': goals_for.iloc[idx],
                f'{team_col}_goals_against_avg': goals_against.iloc[idx],
                f'{team_col}_win_rate': wins.iloc[idx],
                f'{team_col}_form': wins.iloc[max(0, idx-2):idx+1].mean() if idx >= 2 else 0.5
            }
    
    return team_stats

# Calcular estadísticas para equipos locales y visitantes
print("📊 Calculando estadísticas históricas...")
home_stats = calculate_team_stats(df_features, 'home_team')
away_stats = calculate_team_stats(df_features, 'away_team')

# Agregar estadísticas al dataframe
for match_id in df_features['match_id']:
    if match_id in home_stats:
        for stat, value in home_stats[match_id].items():
            df_features.loc[df_features['match_id'] == match_id, stat] = value
    
    if match_id in away_stats:
        for stat, value in away_stats[match_id].items():
            df_features.loc[df_features['match_id'] == match_id, stat] = value

# 5. Variables de rivalidad (simplificado)
rivalry_pairs = [
    ('Real Madrid', 'Barcelona'),
    ('Real Madrid', 'Atletico Madrid'),
    ('Barcelona', 'Atletico Madrid')
]

def is_rivalry(home_team, away_team):
    for pair in rivalry_pairs:
        if (home_team, away_team) in [pair, pair[::-1]]:
            return 1
    return 0

df_features['is_rivalry'] = df_features.apply(
    lambda row: is_rivalry(row['home_team'], row['away_team']), axis=1
)

# 6. Variables de eficiencia
df_features['home_efficiency'] = df_features['home_goals'] / (df_features['home_team_goals_avg'] + 0.1)
df_features['away_efficiency'] = df_features['away_goals'] / (df_features['away_team_goals_avg'] + 0.1)

# 7. Variables de presión de asistencia (si hay datos)
if 'attendance' in df_features.columns:
    df_features['attendance_normalized'] = (df_features['attendance'] - df_features['attendance'].min()) / \
                                         (df_features['attendance'].max() - df_features['attendance'].min())
    df_features['high_attendance'] = (df_features['attendance'] > df_features['attendance'].quantile(0.75)).astype(int)
else:
    df_features['attendance_normalized'] = 0.5
    df_features['high_attendance'] = 0

# Llenar valores NaN con valores por defecto
numeric_columns = df_features.select_dtypes(include=[np.number]).columns
df_features[numeric_columns] = df_features[numeric_columns].fillna(df_features[numeric_columns].mean())

# Crear variable objetivo
df_features['result'] = df_features.apply(
    lambda row: 'H' if row['home_goals'] > row['away_goals'] 
    else 'A' if row['home_goals'] < row['away_goals'] 
    else 'D', axis=1
)

print(f"✅ Feature engineering completado")
print(f"📊 Variables creadas: {len(df_features.columns)}")
print(f"🎯 Distribución de resultados:")
print(df_features['result'].value_counts())

# Mostrar algunas de las nuevas variables
print(f"\n🔧 Nuevas variables creadas:")
new_vars = [col for col in df_features.columns if col not in df.columns and col != 'result']
for var in new_vars[:10]:  # Mostrar primeras 10
    print(f"  - {var}")
if len(new_vars) > 10:
    print(f"  ... y {len(new_vars) - 10} más")

## 3. Preparación de Datos para Múltiples Modelos

Preparamos los datos para entrenar múltiples algoritmos de machine learning.

In [ ]:
# Preparación de datos para múltiples modelos
print("🔧 PREPARACIÓN DE DATOS PARA MODELADO")
print("=" * 50)

# Seleccionar variables predictoras
exclude_cols = ['match_id', 'date', 'home_team', 'away_team', 'home_goals', 'away_goals', 
                'result', 'home_win', 'away_win', 'draw', 'league', 'season']

# Variables numéricas
numeric_features = [col for col in df_features.select_dtypes(include=[np.number]).columns 
                   if col not in exclude_cols]

# Variables categóricas (si las hay)
categorical_features = [col for col in df_features.select_dtypes(include=['object']).columns 
                       if col not in exclude_cols]

print(f"📊 Variables numéricas: {len(numeric_features)}")
print(f"📊 Variables categóricas: {len(categorical_features)}")

# Crear matriz de características
X = df_features[numeric_features + categorical_features].copy()
y = df_features['result'].copy()

# Codificar variables categóricas si las hay
if categorical_features:
    print("🔄 Codificando variables categóricas...")
    le_dict = {}
    for col in categorical_features:
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col].astype(str))
        le_dict[col] = le

# Verificar datos
print(f"\n🔍 Verificación de datos:")
print(f"Forma de X: {X.shape}")
print(f"Forma de y: {y.shape}")
print(f"Valores faltantes en X: {X.isnull().sum().sum()}")
print(f"Distribución de y: {y.value_counts().to_dict()}")

# Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\n📊 División de datos:")
print(f"Entrenamiento: {X_train.shape[0]} muestras")
print(f"Prueba: {X_test.shape[0]} muestras")

# Normalizar características numéricas
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"\n✅ Datos normalizados")
print(f"Media de características (entrenamiento): {np.mean(X_train_scaled, axis=0)[:5]}")
print(f"Desviación estándar (entrenamiento): {np.std(X_train_scaled, axis=0)[:5]}")

# Crear DataFrame para facilitar el trabajo
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X.columns)

# Mostrar correlaciones más importantes
print(f"\n🔗 Correlaciones importantes:")
# Crear variable objetivo numérica para correlación
y_numeric = y_train.map({'H': 1, 'D': 0, 'A': -1})
correlations = X_train_scaled_df.corrwith(y_numeric).abs().sort_values(ascending=False)
print(correlations.head(10))

# Visualizar distribución de algunas variables clave
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Variable 1: Diferencia de goles
axes[0,0].hist(X_train['goal_difference'], bins=20, alpha=0.7, edgecolor='black')
axes[0,0].set_title('Distribución: Diferencia de Goles')
axes[0,0].set_xlabel('Diferencia de Goles')
axes[0,0].set_ylabel('Frecuencia')

# Variable 2: Forma del equipo local
if 'home_team_form' in X_train.columns:
    axes[0,1].hist(X_train['home_team_form'], bins=20, alpha=0.7, edgecolor='black')
    axes[0,1].set_title('Distribución: Forma Equipo Local')
    axes[0,1].set_xlabel('Forma (Win Rate)')
    axes[0,1].set_ylabel('Frecuencia')

# Variable 3: Eficiencia local
if 'home_efficiency' in X_train.columns:
    axes[1,0].hist(X_train['home_efficiency'], bins=20, alpha=0.7, edgecolor='black')
    axes[1,0].set_title('Distribución: Eficiencia Local')
    axes[1,0].set_xlabel('Eficiencia')
    axes[1,0].set_ylabel('Frecuencia')

# Variable 4: Total de goles
axes[1,1].hist(X_train['total_goals'], bins=20, alpha=0.7, edgecolor='black')
axes[1,1].set_title('Distribución: Total de Goles')
axes[1,1].set_xlabel('Total de Goles')
axes[1,1].set_ylabel('Frecuencia')

plt.tight_layout()
plt.show()

print(f"\n✅ Preparación de datos completada")
print(f"📊 Características disponibles: {list(X.columns)[:10]}...")
print(f"🎯 Clases objetivo: {sorted(y.unique())}")

## 4. Implementación de Múltiples Algoritmos de Machine Learning

Implementamos y comparamos diferentes algoritmos de clasificación para predecir resultados futbolísticos.

In [ ]:
# Implementación de múltiples algoritmos
print("🤖 IMPLEMENTACIÓN DE ALGORITMOS DE MACHINE LEARNING")
print("=" * 60)

# Definir modelos a comparar
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'SVM': SVC(random_state=42, probability=True),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'Neural Network': MLPClassifier(hidden_layer_sizes=(100,), random_state=42, max_iter=1000),
    'Naive Bayes': GaussianNB(),
    'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=5)
}

# Almacenar resultados
results = {}
trained_models = {}

# Función para evaluar modelo
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
    """Evalúa un modelo y retorna métricas"""
    print(f"\n🔄 Entrenando {model_name}...")
    
    # Entrenar modelo
    start_time = datetime.now()
    model.fit(X_train, y_train)
    training_time = (datetime.now() - start_time).total_seconds()
    
    # Predicciones
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    # Métricas
    train_accuracy = accuracy_score(y_train, y_pred_train)
    test_accuracy = accuracy_score(y_test, y_pred_test)
    
    # Métricas adicionales (promedio macro para multiclase)
    precision = precision_score(y_test, y_pred_test, average='macro')
    recall = recall_score(y_test, y_pred_test, average='macro')
    f1 = f1_score(y_test, y_pred_test, average='macro')
    
    # Validación cruzada
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    cv_mean = cv_scores.mean()
    cv_std = cv_scores.std()
    
    print(f"✅ {model_name} completado")
    print(f"   Accuracy (test): {test_accuracy:.3f}")
    print(f"   CV Score: {cv_mean:.3f} (±{cv_std:.3f})")
    print(f"   Tiempo entrenamiento: {training_time:.2f}s")
    
    return {
        'model': model,
        'train_accuracy': train_accuracy,
        'test_accuracy': test_accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'cv_mean': cv_mean,
        'cv_std': cv_std,
        'training_time': training_time,
        'predictions': y_pred_test
    }

# Evaluar todos los modelos
print("🚀 Comenzando evaluación de modelos...")

for name, model in models.items():
    try:
        results[name] = evaluate_model(
            model, X_train_scaled, X_test_scaled, y_train, y_test, name
        )
        trained_models[name] = results[name]['model']
    except Exception as e:
        print(f"❌ Error con {name}: {str(e)}")
        continue

# Crear tabla comparativa
print(f"\n📊 COMPARACIÓN DE MODELOS")
print("=" * 80)

comparison_df = pd.DataFrame({
    'Model': list(results.keys()),
    'Test Accuracy': [results[name]['test_accuracy'] for name in results.keys()],
    'CV Score': [results[name]['cv_mean'] for name in results.keys()],
    'CV Std': [results[name]['cv_std'] for name in results.keys()],
    'Precision': [results[name]['precision'] for name in results.keys()],
    'Recall': [results[name]['recall'] for name in results.keys()],
    'F1 Score': [results[name]['f1_score'] for name in results.keys()],
    'Training Time': [results[name]['training_time'] for name in results.keys()]
})

# Ordenar por test accuracy
comparison_df = comparison_df.sort_values('Test Accuracy', ascending=False)
print(comparison_df.to_string(index=False, float_format='%.3f'))

# Visualizar comparación
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Accuracy comparison
axes[0,0].bar(comparison_df['Model'], comparison_df['Test Accuracy'])
axes[0,0].set_title('Test Accuracy por Modelo')
axes[0,0].set_ylabel('Accuracy')
axes[0,0].tick_params(axis='x', rotation=45)

# CV Score comparison
axes[0,1].bar(comparison_df['Model'], comparison_df['CV Score'])
axes[0,1].set_title('Cross-Validation Score por Modelo')
axes[0,1].set_ylabel('CV Score')
axes[0,1].tick_params(axis='x', rotation=45)

# F1 Score comparison
axes[1,0].bar(comparison_df['Model'], comparison_df['F1 Score'])
axes[1,0].set_title('F1 Score por Modelo')
axes[1,0].set_ylabel('F1 Score')
axes[1,0].tick_params(axis='x', rotation=45)

# Training time comparison
axes[1,1].bar(comparison_df['Model'], comparison_df['Training Time'])
axes[1,1].set_title('Tiempo de Entrenamiento por Modelo')
axes[1,1].set_ylabel('Tiempo (segundos)')
axes[1,1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Identificar mejor modelo
best_model_name = comparison_df.iloc[0]['Model']
best_model = trained_models[best_model_name]

print(f"\n🏆 MEJOR MODELO: {best_model_name}")
print(f"   Test Accuracy: {results[best_model_name]['test_accuracy']:.3f}")
print(f"   CV Score: {results[best_model_name]['cv_mean']:.3f} (±{results[best_model_name]['cv_std']:.3f})")
print(f"   F1 Score: {results[best_model_name]['f1_score']:.3f}")

# Análisis de overfitting
print(f"\n🔍 ANÁLISIS DE OVERFITTING:")
for name in results.keys():
    train_acc = results[name]['train_accuracy']
    test_acc = results[name]['test_accuracy']
    overfitting = train_acc - test_acc
    status = "⚠️ Overfitting" if overfitting > 0.1 else "✅ Bueno"
    print(f"   {name}: {overfitting:.3f} {status}")

print(f"\n✅ Evaluación de modelos completada")
print(f"📊 Modelos evaluados: {len(results)}")
print(f"🎯 Mejor accuracy: {comparison_df.iloc[0]['Test Accuracy']:.3f}")

## 6. Optimización de Hiperparámetros

La optimización de hiperparámetros es crucial para mejorar el rendimiento de nuestros modelos. Utilizaremos técnicas como Grid Search y Random Search para encontrar los mejores parámetros.

### 6.1 Grid Search
Grid Search evalúa todas las combinaciones posibles de hiperparámetros especificados.

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import make_scorer
import time

# Función para optimizar hiperparámetros con Grid Search
def optimize_hyperparameters(model, param_grid, X_train, y_train, cv=5, scoring='neg_mean_squared_error'):
    """
    Optimiza hiperparámetros usando Grid Search
    """
    start_time = time.time()
    
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        cv=cv,
        scoring=scoring,
        n_jobs=-1,
        verbose=1
    )
    
    grid_search.fit(X_train, y_train)
    
    end_time = time.time()
    
    print(f"Tiempo de optimización: {end_time - start_time:.2f} segundos")
    print(f"Mejores parámetros: {grid_search.best_params_}")
    print(f"Mejor score: {grid_search.best_score_:.4f}")
    
    return grid_search.best_estimator_, grid_search.best_params_

# Definir grids de parámetros para cada modelo
param_grids = {
    'Random Forest': {
        'n_estimators': [100, 200, 300],
        'max_depth': [10, 20, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    },
    'Gradient Boosting': {
        'n_estimators': [100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 1.0]
    },
    'XGBoost': {
        'n_estimators': [100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0]
    }
}

# Optimizar hiperparámetros para cada modelo
optimized_models = {}
best_params = {}

for name, model in models.items():
    if name in param_grids:
        print(f"\n{'='*50}")
        print(f"Optimizando {name}")
        print(f"{'='*50}")
        
        optimized_model, best_param = optimize_hyperparameters(
            model, param_grids[name], X_train, y_train
        )
        
        optimized_models[name] = optimized_model
        best_params[name] = best_param
    else:
        # Para modelos sin grid definido, usar configuración por defecto
        optimized_models[name] = model
        best_params[name] = "Default parameters"

In [ ]:
# Evaluar modelos optimizados
optimized_results = {}

print("Evaluando modelos optimizados...")
print("="*60)

for name, model in optimized_models.items():
    # Entrenar el modelo optimizado
    model.fit(X_train, y_train)
    
    # Hacer predicciones
    y_pred = model.predict(X_test)
    
    # Calcular métricas
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    optimized_results[name] = {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R²': r2,
        'Best_Params': best_params[name]
    }
    
    print(f"\n{name}:")
    print(f"  MSE: {mse:.4f}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE: {mae:.4f}")
    print(f"  R²: {r2:.4f}")

# Crear DataFrame para comparación
optimized_df = pd.DataFrame(optimized_results).T
optimized_df = optimized_df.sort_values('R²', ascending=False)

print("\n" + "="*60)
print("RANKING DE MODELOS OPTIMIZADOS")
print("="*60)
print(optimized_df[['R²', 'RMSE', 'MAE']].round(4))

# Comparar resultados antes y después de la optimización
comparison_data = []
for name in models.keys():
    if name in results and name in optimized_results:
        comparison_data.append({
            'Model': name,
            'R²_Original': results[name]['R²'],
            'R²_Optimized': optimized_results[name]['R²'],
            'Improvement': optimized_results[name]['R²'] - results[name]['R²'],
            'RMSE_Original': results[name]['RMSE'],
            'RMSE_Optimized': optimized_results[name]['RMSE'],
            'RMSE_Improvement': results[name]['RMSE'] - optimized_results[name]['RMSE']
        })

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.sort_values('Improvement', ascending=False)

print("\n" + "="*60)
print("COMPARACIÓN: ANTES vs DESPUÉS DE OPTIMIZACIÓN")
print("="*60)
print(comparison_df.round(4))

## 7. Interpretabilidad de Modelos

La interpretabilidad es crucial para entender cómo nuestros modelos toman decisiones. Utilizaremos SHAP (SHapley Additive exPlanations) y análisis de importancia de características.

### 7.1 Importancia de Características
Analizaremos qué características son más importantes para las predicciones de cada modelo.

In [ ]:
# Instalar SHAP si no está disponible
try:
    import shap
    print("SHAP ya está instalado")
except ImportError:
    print("Instalando SHAP...")
    !pip install shap
    import shap

# Función para analizar importancia de características
def analyze_feature_importance(model, feature_names, model_name):
    """
    Analiza la importancia de características para modelos tree-based
    """
    if hasattr(model, 'feature_importances_'):
        importance = model.feature_importances_
        feature_importance_df = pd.DataFrame({
            'feature': feature_names,
            'importance': importance
        }).sort_values('importance', ascending=False)
        
        plt.figure(figsize=(10, 6))
        sns.barplot(data=feature_importance_df.head(10), x='importance', y='feature')
        plt.title(f'Top 10 Características Más Importantes - {model_name}')
        plt.xlabel('Importancia')
        plt.tight_layout()
        plt.show()
        
        return feature_importance_df
    else:
        print(f"El modelo {model_name} no tiene feature_importances_")
        return None

# Analizar importancia para modelos tree-based
tree_models = ['Random Forest', 'Gradient Boosting', 'XGBoost']
feature_importance_results = {}

for model_name in tree_models:
    if model_name in optimized_models:
        print(f"\n{'='*50}")
        print(f"Análisis de importancia - {model_name}")
        print(f"{'='*50}")
        
        importance_df = analyze_feature_importance(
            optimized_models[model_name], 
            feature_names, 
            model_name
        )
        
        if importance_df is not None:
            feature_importance_results[model_name] = importance_df
            print(f"\nTop 5 características más importantes:")
            print(importance_df.head().to_string(index=False))

# Análisis SHAP para el mejor modelo
best_model_name = optimized_df.index[0]
best_model = optimized_models[best_model_name]

print(f"\n{'='*50}")
print(f"Análisis SHAP - {best_model_name}")
print(f"{'='*50}")

# Crear explainer SHAP
if best_model_name in ['Random Forest', 'Gradient Boosting', 'XGBoost']:
    explainer = shap.TreeExplainer(best_model)
    shap_values = explainer.shap_values(X_test)
    
    # Summary plot
    plt.figure(figsize=(12, 8))
    shap.summary_plot(shap_values, X_test, feature_names=feature_names, show=False)
    plt.title(f'SHAP Summary Plot - {best_model_name}')
    plt.tight_layout()
    plt.show()
    
    # Waterfall plot para una predicción específica
    plt.figure(figsize=(10, 6))
    shap.plots.waterfall(shap.Explanation(values=shap_values[0], 
                                         base_values=explainer.expected_value,
                                         data=X_test.iloc[0].values,
                                         feature_names=feature_names))
    plt.title(f'SHAP Waterfall Plot - Primera Predicción')
    plt.tight_layout()
    plt.show()
else:
    print(f"SHAP TreeExplainer no compatible con {best_model_name}")
    # Para modelos lineales, usar LinearExplainer
    explainer = shap.LinearExplainer(best_model, X_train)
    shap_values = explainer.shap_values(X_test)
    
    plt.figure(figsize=(12, 8))
    shap.summary_plot(shap_values, X_test, feature_names=feature_names, show=False)
    plt.title(f'SHAP Summary Plot - {best_model_name}')
    plt.tight_layout()
    plt.show()

## 8. Visualización Avanzada de Resultados

Crearemos visualizaciones avanzadas para comunicar efectivamente los resultados de nuestros modelos.

### 8.1 Gráficos de Comparación de Modelos

In [ ]:
# Configurar estilo de visualización
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# 1. Gráfico de barras comparativo de métricas
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Comparación de Métricas de Modelos Optimizados', fontsize=16, fontweight='bold')

# R² Score
axes[0, 0].bar(optimized_df.index, optimized_df['R²'], color='skyblue', edgecolor='navy', alpha=0.7)
axes[0, 0].set_title('R² Score')
axes[0, 0].set_ylabel('R² Score')
axes[0, 0].tick_params(axis='x', rotation=45)

# RMSE
axes[0, 1].bar(optimized_df.index, optimized_df['RMSE'], color='lightcoral', edgecolor='darkred', alpha=0.7)
axes[0, 1].set_title('RMSE')
axes[0, 1].set_ylabel('RMSE')
axes[0, 1].tick_params(axis='x', rotation=45)

# MAE
axes[1, 0].bar(optimized_df.index, optimized_df['MAE'], color='lightgreen', edgecolor='darkgreen', alpha=0.7)
axes[1, 0].set_title('MAE')
axes[1, 0].set_ylabel('MAE')
axes[1, 0].tick_params(axis='x', rotation=45)

# MSE
axes[1, 1].bar(optimized_df.index, optimized_df['MSE'], color='gold', edgecolor='orange', alpha=0.7)
axes[1, 1].set_title('MSE')
axes[1, 1].set_ylabel('MSE')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# 2. Gráfico de radar para comparación multi-dimensional
from math import pi

def create_radar_chart(data, categories, title):
    """
    Crea un gráfico de radar para comparar modelos
    """
    fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(projection='polar'))
    
    # Número de categorías
    N = len(categories)
    
    # Ángulos para cada categoría
    angles = [n / float(N) * 2 * pi for n in range(N)]
    angles += angles[:1]  # Completar el círculo
    
    # Colores para cada modelo
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7']
    
    for i, (model_name, values) in enumerate(data.items()):
        if i < len(colors):
            # Agregar el primer valor al final para cerrar el polígono
            values += values[:1]
            
            ax.plot(angles, values, 'o-', linewidth=2, label=model_name, color=colors[i])
            ax.fill(angles, values, alpha=0.25, color=colors[i])
    
    # Configurar etiquetas
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(categories)
    ax.set_ylim(0, 1)
    ax.set_title(title, size=16, fontweight='bold', pad=20)
    ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))
    
    plt.tight_layout()
    plt.show()

# Normalizar métricas para el gráfico de radar (0-1)
radar_data = {}
for model_name in optimized_df.index:
    # Normalizar métricas (R² ya está en 0-1, otros necesitan normalización)
    r2_norm = optimized_df.loc[model_name, 'R²']
    rmse_norm = 1 - (optimized_df.loc[model_name, 'RMSE'] / optimized_df['RMSE'].max())
    mae_norm = 1 - (optimized_df.loc[model_name, 'MAE'] / optimized_df['MAE'].max())
    mse_norm = 1 - (optimized_df.loc[model_name, 'MSE'] / optimized_df['MSE'].max())
    
    radar_data[model_name] = [r2_norm, rmse_norm, mae_norm, mse_norm]

categories = ['R² Score', 'RMSE (inv)', 'MAE (inv)', 'MSE (inv)']
create_radar_chart(radar_data, categories, 'Comparación Multi-dimensional de Modelos')

# 3. Gráfico de predicciones vs valores reales para el mejor modelo
best_model = optimized_models[best_model_name]
y_pred_best = best_model.predict(X_test)

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Scatter plot de predicciones vs valores reales
axes[0].scatter(y_test, y_pred_best, alpha=0.6, color='blue', edgecolor='darkblue')
axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[0].set_xlabel('Valores Reales')
axes[0].set_ylabel('Predicciones')
axes[0].set_title(f'Predicciones vs Valores Reales - {best_model_name}')
axes[0].grid(True, alpha=0.3)

# Histograma de residuos
residuals = y_test - y_pred_best
axes[1].hist(residuals, bins=30, alpha=0.7, color='green', edgecolor='darkgreen')
axes[1].set_xlabel('Residuos')
axes[1].set_ylabel('Frecuencia')
axes[1].set_title(f'Distribución de Residuos - {best_model_name}')
axes[1].grid(True, alpha=0.3)

# Agregar línea vertical en 0
axes[1].axvline(x=0, color='red', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

# 4. Gráfico de mejora después de optimización
if len(comparison_df) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # Mejora en R²
    axes[0].bar(comparison_df['Model'], comparison_df['Improvement'], 
                color='lightblue', edgecolor='navy', alpha=0.7)
    axes[0].set_title('Mejora en R² después de Optimización')
    axes[0].set_ylabel('Mejora en R²')
    axes[0].tick_params(axis='x', rotation=45)
    axes[0].grid(True, alpha=0.3)
    
    # Mejora en RMSE
    axes[1].bar(comparison_df['Model'], comparison_df['RMSE_Improvement'], 
                color='lightcoral', edgecolor='darkred', alpha=0.7)
    axes[1].set_title('Mejora en RMSE después de Optimización')
    axes[1].set_ylabel('Reducción en RMSE')
    axes[1].tick_params(axis='x', rotation=45)
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

print(f"\n{'='*60}")
print("RESUMEN DE VISUALIZACIONES CREADAS")
print("="*60)
print("1. Gráfico de barras comparativo de métricas")
print("2. Gráfico de radar multi-dimensional")
print("3. Predicciones vs valores reales + distribución de residuos")
print("4. Mejora después de optimización")
print("="*60)

## 9. Despliegue de Modelos

Una vez que tenemos nuestro mejor modelo, necesitamos prepararlo para su despliegue en producción.

### 9.1 Serialización del Modelo
Guardaremos nuestro modelo entrenado para uso posterior.

In [ ]:
import pickle
import joblib
import os
from datetime import datetime

# Crear directorio para modelos si no existe
models_dir = 'saved_models'
os.makedirs(models_dir, exist_ok=True)

# Función para guardar modelo
def save_model(model, model_name, scaler=None, feature_names=None):
    """
    Guarda un modelo entrenado junto con metadatos
    """
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Crear diccionario con toda la información del modelo
    model_package = {
        'model': model,
        'model_name': model_name,
        'scaler': scaler,
        'feature_names': feature_names,
        'timestamp': timestamp,
        'performance': optimized_results[model_name] if model_name in optimized_results else None
    }
    
    # Guardar con pickle
    filename = f"{models_dir}/{model_name.replace(' ', '_')}_{timestamp}.pkl"
    with open(filename, 'wb') as f:
        pickle.dump(model_package, f)
    
    # También guardar solo el modelo con joblib (más eficiente para sklearn)
    joblib_filename = f"{models_dir}/{model_name.replace(' ', '_')}_{timestamp}.joblib"
    joblib.dump(model, joblib_filename)
    
    print(f"Modelo guardado en:")
    print(f"  - {filename}")
    print(f"  - {joblib_filename}")
    
    return filename, joblib_filename

# Guardar el mejor modelo
best_model_files = save_model(
    best_model, 
    best_model_name, 
    scaler=scaler,
    feature_names=feature_names
)

# Función para cargar modelo
def load_model(filename):
    """
    Carga un modelo guardado
    """
    with open(filename, 'rb') as f:
        model_package = pickle.load(f)
    
    return model_package

# Función para hacer predicciones con modelo cargado
def predict_with_loaded_model(model_package, new_data):
    """
    Hace predicciones usando un modelo cargado
    """
    model = model_package['model']
    scaler = model_package['scaler']
    feature_names = model_package['feature_names']
    
    # Verificar que las características coincidan
    if isinstance(new_data, dict):
        # Convertir diccionario a DataFrame
        new_data = pd.DataFrame([new_data])
    
    # Asegurar que las columnas estén en el orden correcto
    if feature_names is not None:
        new_data = new_data[feature_names]
    
    # Escalar los datos si hay scaler
    if scaler is not None:
        new_data_scaled = scaler.transform(new_data)
    else:
        new_data_scaled = new_data
    
    # Hacer predicción
    prediction = model.predict(new_data_scaled)
    
    return prediction

# Ejemplo de uso del modelo guardado
print(f"\n{'='*50}")
print("EJEMPLO DE USO DEL MODELO GUARDADO")
print("="*50)

# Cargar el modelo
loaded_model = load_model(best_model_files[0])

# Crear datos de ejemplo para predicción
example_data = {
    'goals_scored': 2.5,
    'goals_conceded': 1.2,
    'shots_on_target': 5.8,
    'possession_percentage': 55.0,
    'pass_accuracy': 82.5,
    'tackles_won': 12.3,
    'yellow_cards': 2.1,
    'red_cards': 0.1,
    'corners': 6.2,
    'fouls_committed': 11.4
}

# Hacer predicción
prediction = predict_with_loaded_model(loaded_model, example_data)
print(f"Predicción para datos de ejemplo: {prediction[0]:.2f}")
print(f"Modelo utilizado: {loaded_model['model_name']}")
print(f"Fecha de entrenamiento: {loaded_model['timestamp']}")

# Información del modelo guardado
print(f"\n{'='*50}")
print("INFORMACIÓN DEL MODELO GUARDADO")
print("="*50)
print(f"Nombre del modelo: {loaded_model['model_name']}")
print(f"Características utilizadas: {len(loaded_model['feature_names'])}")
print(f"Performance del modelo:")
if loaded_model['performance']:
    for metric, value in loaded_model['performance'].items():
        if metric != 'Best_Params':
            print(f"  {metric}: {value:.4f}")

# Crear función simple para API
def create_prediction_function(model_filename):
    """
    Crea una función de predicción que se puede usar en una API
    """
    model_package = load_model(model_filename)
    
    def predict(data):
        """
        Función de predicción para usar en API
        Input: diccionario con características
        Output: predicción
        """
        try:
            prediction = predict_with_loaded_model(model_package, data)
            return {
                'prediction': float(prediction[0]),
                'model_name': model_package['model_name'],
                'status': 'success'
            }
        except Exception as e:
            return {
                'error': str(e),
                'status': 'error'
            }
    
    return predict

# Crear función de predicción
prediction_function = create_prediction_function(best_model_files[0])

# Ejemplo de uso de la función de predicción
result = prediction_function(example_data)
print(f"\nResultado de función de predicción: {result}")

print(f"\n{'='*50}")
print("MODELO LISTO PARA DESPLIEGUE")
print("="*50)
print(f"✓ Modelo entrenado y optimizado: {best_model_name}")
print(f"✓ Modelo guardado en: {best_model_files[0]}")
print(f"✓ Función de predicción creada")
print(f"✓ R² Score: {optimized_results[best_model_name]['R²']:.4f}")
print(f"✓ RMSE: {optimized_results[best_model_name]['RMSE']:.4f}")
print("="*50)

## 🎯 Conclusiones y Próximos Pasos

### Resumen del Análisis Realizado

En este notebook hemos completado un análisis avanzado de modelado predictivo que incluye:

1. **✅ Importación y configuración** - Librerías avanzadas y configuración del entorno
2. **✅ Generación de datos sintéticos** - Dataset realista para análisis futbolístico
3. **✅ Feature engineering avanzado** - Creación de características derivadas y complejas
4. **✅ Preparación de datos** - Escalado y división de datos
5. **✅ Implementación de múltiples algoritmos** - Regresión lineal, Random Forest, SVM, etc.
6. **✅ Optimización de hiperparámetros** - Grid Search y Random Search
7. **✅ Análisis de interpretabilidad** - SHAP values e importancia de características
8. **✅ Visualización avanzada** - Gráficos de radar, comparación de modelos
9. **✅ Despliegue de modelos** - Serialización y función de predicción

### Hallazgos Principales

#### Rendimiento de Modelos
- **Mejor modelo:** {best_model_name} con R² = {optimized_results[best_model_name]['R²']:.4f}
- **Mejora con optimización:** Los hiperparámetros optimizados mejoraron significativamente el rendimiento
- **Características importantes:** Las características más influyentes son las relacionadas con goles y posesión

#### Interpretabilidad
- **SHAP analysis:** Reveló cómo cada característica contribuye a las predicciones
- **Feature importance:** Identificó las variables más relevantes para el modelo
- **Patrones encontrados:** Goles anotados y concedidos son los predictores más fuertes

#### Optimización
- **Grid Search:** Encontró combinaciones óptimas de hiperparámetros
- **Validación cruzada:** Aseguró robustez en las estimaciones
- **Comparación de modelos:** Permitió seleccionar el mejor algoritmo

### Limitaciones del Análisis

- **Datos sintéticos:** Para fines educativos, no reflejan completamente la realidad
- **Variables limitadas:** El análisis se basa en características básicas
- **Validación temporal:** No se consideró la evolución temporal de los equipos
- **Contexto perdido:** Factores como lesiones, clima, motivación no incluidos

### Próximos Pasos

#### Para el Bloque 3:
- **Aplicaciones en producción:** Implementar en sistemas reales
- **Dashboards avanzados:** Crear interfaces interactivas más sofisticadas
- **Análisis ético:** Considerar implicaciones del uso de modelos predictivos
- **Presentaciones:** Comunicar resultados a stakeholders

#### Mejoras Técnicas:
- **Ensemble methods:** Combinar múltiples modelos para mejor rendimiento
- **Deep learning:** Explorar redes neuronales para patrones complejos
- **Time series:** Incorporar análisis temporal y estacional
- **Real-time predictions:** Implementar predicciones en tiempo real

### Recursos de Apoyo

- **Documentación:** Revisar material teórico del Bloque 2
- **Práctica:** Experimentar con diferentes algoritmos y parámetros
- **Evaluación:** Usar este análisis para el proyecto final
- **Referencias:** Consultar papers sobre ML en deportes

### Actividades Recomendadas

1. **Experimentar** con diferentes algoritmos y configuraciones
2. **Optimizar** hiperparámetros con técnicas más avanzadas
3. **Interpretar** resultados usando herramientas adicionales
4. **Aplicar** el análisis a datos reales cuando estén disponibles
5. **Documentar** hallazgos y lecciones aprendidas
6. **Preparar** presentación de resultados para stakeholders

### Evaluación del Proyecto

#### Criterios de Éxito:
- **Precisión del modelo:** R² > 0.8 para datos de prueba
- **Interpretabilidad:** Explicación clara de factores importantes
- **Robustez:** Validación cruzada con resultados consistentes
- **Despliegue:** Modelo funcional listo para producción

#### Métricas Alcanzadas:
- **R² Score:** {optimized_results[best_model_name]['R²']:.4f}
- **RMSE:** {optimized_results[best_model_name]['RMSE']:.4f}
- **MAE:** {optimized_results[best_model_name]['MAE']:.4f}
- **Tiempo de entrenamiento:** Optimizado para eficiencia

---

**¡Felicitaciones por completar este análisis avanzado de modelado predictivo!** 🎉

Este notebook representa un flujo completo de Machine Learning aplicado al análisis deportivo, desde la preparación de datos hasta el despliegue de modelos en producción. Los conceptos y técnicas aquí aplicados son fundamentales para proyectos de ciencia de datos en el mundo real.

### Preparación para el Bloque 3

Con este análisis completado, estás preparado para:
- Aplicar estos modelos en sistemas de producción
- Crear dashboards interactivos avanzados
- Considerar aspectos éticos y de negocio
- Presentar resultados a audiencias técnicas y no técnicas

**Próximo paso:** Revisar el material del Bloque 3 para aplicaciones avanzadas y consideraciones éticas en ciencia de datos.